In [2]:
import sys
sys.path.append(r"E:\labs\ML Lab\ML Lab Spring 25\timeseires")

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [4]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [5]:
def CNN():
    input_data = Input(shape=(time_steps, num_features))
    x1 = Conv1D(16, 2, activation="relu")(input_data)
    x2 = Conv1D(16, 2, activation="relu")(x1)
    flatten = Flatten()(x2)
    output_data = Dense(1)(flatten)
    model = Model(input_data, output_data)
    return model

In [6]:
model1 = CNN()
model1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 21)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 23, 16)            688       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 16)            528       
_________________________________________________________________
flatten (Flatten)            (None, 352)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 353       
Total params: 1,569
Trainable params: 1,569
Non-trainable params: 0
_________________________________________________________________


In [7]:
tensorflow.keras.utils.plot_model(model1 )

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [8]:
checkpoints = r'E:\labs\ml lab\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:\labs\ml lab\ML Lab\lab8'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
import os
from keras.callbacks import ModelCheckpoint

# Define necessary paths and variables
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

FIG_PATH = os.path.sep.join([output_dir, "training_plot.png"])
JSON_PATH = os.path.sep.join([output_dir, "training_history.json"])
checkpoints = os.path.join(r'E:\labs\ml lab\lab8', 'best_model.h5')

start_epoch = 0  # Change this if you're resuming from a checkpoint

# Construct the callback to save only the *best* model to disk based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                                   monitor="val_loss",
                                   save_best_only=True, 
                                   verbose=1)

# Make sure the TrainingMonitor class is implemented/imported properly
TrainingMonitor1 = TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1, TrainingMonitor1]


In [10]:
from keras.models import load_model
from keras.optimizers import Adam
import keras.backend as K

# Define your model path
model_path =r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab8\best_model.h5'

# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if not os.path.exists(model_path):  # If the model file doesn't exist, start fresh
    print("[INFO] compiling model...")
    model = CNN()  # define your CNN() model earlier
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

# otherwise, load the checkpoint from disk
else:
    print(f"[INFO] loading model from {model_path}...")
    model = load_model(model_path)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))


[INFO] compiling model...


In [11]:
import os
path_dataset =r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\FYP DETAILS\dada sets\AEP\AEP'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [12]:
time_steps=24
num_features=21

In [13]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 11.499531507492065 sec


In [15]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0194 - mae: 0.0194 - mape: 1177.2150
Epoch 00001: val_loss improved from inf to 0.01625, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab8\best_model.h5
2653/2653 [==============================] - 40s 15ms/step - loss: 0.0194 - mae: 0.0194 - mape: 1177.2150 - val_loss: 0.0163 - val_mae: 0.0163 - val_mape: 7.6260
Epoch 2/10
2650/2653 [============================>.] - ETA: 0s - loss: 0.0148 - mae: 0.0148 - mape: 656.6852
Epoch 00002: val_loss improved from 0.01625 to 0.01446, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab8\best_model.h5
2653/2653 [==============================] - 41s 15ms/step - loss: 0.0148 - mae: 0.0148 - mape: 656.0554 - val_loss: 0.0145 - val_mae: 0.0145 - val_mape: 7.7883
Epoch 3/10
2652/2653 [============================>.] - ETA: 0s - loss: 0.0127 - mae: 0.0127 - mape: 665.2752
Epoch 00003: val_loss improved from 0.01446 to 0.01080

In [16]:

model = load_model(r'E:\labs\ml lab\lab8\best_model.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 125.42
Median Absolute Error (MedAE): 100.79
Mean Squared Error (MSE): 26815.19
Root Mean Squared Error (RMSE): 163.75
Mean Absolute Percentage Error (MAPE): 0.86 %
Median Absolute Percentage Error (MDAPE): 0.69 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [18]:
checkpoints = r'E:\labs\ml lab\lab8\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\labs\ml lab\lab8\lab8\best_model.h5'
start_epoch= 12

In [19]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab8\best_model.h5...


[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [20]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/5
2652/2653 [============================>.] - ETA: 0s - loss: 0.0070 - mean_absolute_error: 0.0070 - mean_absolute_percentage_error: 165.4842
Epoch 00001: val_loss improved from inf to 0.00705, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab8\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 45s 17ms/step - loss: 0.0070 - mean_absolute_error: 0.0070 - mean_absolute_percentage_error: 165.4495 - val_loss: 0.0070 - val_mean_absolute_error: 0.0070 - val_mean_absolute_percentage_error: 3.1621
Epoch 2/5
2652/2653 [============================>.] - ETA: 0s - loss: 0.0069 - mean_absolute_error: 0.0069 - mean_absolute_percentage_error: 105.0119 ETA: 0s - loss: 0.0069 - mean_absolute_error: 0.0069 - mean_absol
Epoch 00002: val_loss did not improve from 0.00705
2653/2653 [==============================] - 39s 15ms/step - loss: 0.0069 - mean_absolute_error: 0.0069 - mean_absolute_percentage_error: 104.9902 - val_loss: 0.0073 - val_mean_absolu

In [21]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab8\E2-cp-0005-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 109.03
Median Absolute Error (MedAE): 86.94
Mean Squared Error (MSE): 20713.59
Root Mean Squared Error (RMSE): 143.92
Mean Absolute Percentage Error (MAPE): 0.75 %
Median Absolute Percentage Error (MDAPE): 0.6 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
